In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import re

In [2]:
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_rows',500)

In [3]:
game = pd.read_csv("game_summary.csv")

In [4]:
play = pd.read_csv("play_by_play.csv")

In [5]:
play = play.drop(['eventnum', 'eventmsgactiontype', 'period',
       'wctimestring', 'pctimestring', 'neutraldescription', 'person1type',
       'player1_team_id', 'player1_team_city',
       'player1_team_nickname', 'player1_team_abbreviation', 'person2type',
       'player2_team_id', 'player2_team_city',
       'player2_team_nickname', 'player2_team_abbreviation', 'person3type',
       'player3_team_id', 'player3_team_city',
       'player3_team_nickname', 'player3_team_abbreviation',
       'video_available_flag'], axis = 1)
play

,game_id,eventmsgtype,homedescription,visitordescription,score,scoremargin,player1_id,player1_name,player2_id,player2_name,player3_id,player3_name
0,29600009,12,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN
1,29600009,10,Jump Ball Olajuwon vs. Polynice: Tip to Williamson,NaN,NaN,NaN,165,Hakeem Olajuwon,178,Olden Polynice,722,Corliss Williamson
2,29600009,5,NaN,Williamson Traveling Turnover (P1.T1),NaN,NaN,722,Corliss Williamson,0,NaN,0,NaN
3,29600009,2,MISS Olajuwon 13' Jump Shot,NaN,NaN,NaN,165,Hakeem Olajuwon,0,NaN,0,NaN
4,29600009,4,Drexler REBOUND (Off:1 Def:0),NaN,NaN,NaN,17,Clyde Drexler,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13227006,22201014,8,SUB: Porter Jr. FOR Smith,NaN,NaN,NaN,202397,Ish Smith,1629008,Michael Porter Jr.,0,NaN
13227007,22201014,8,NaN,SUB: Claxton FOR Johnson,NaN,NaN,1629661,Cameron Johnson,1629651,Nic Claxton,0,NaN
13227008,22201014,2,MISS Jokic 25' 3PT Turnaround Jump Shot,NaN,NaN,NaN,203999,Nikola Jokic,0,NaN,0,NaN
13227009,22201014,4,NUGGETS Rebound,NaN,NaN,NaN,1610612743,NaN,0,NaN,0,NaN


In [6]:
game_dict = {'1946' : [], '1947' : [], '1948' : [], '1949' : [], '1950' : [], '1951' : [], '1952' : [], '1953' : [], '1954' : [], '1955' : [], '1956' : [], '1957' : [], '1958' : [], 
             '1959' : [], '1960' : [], '1961' : [], '1962' : [], '1963' : [], '1964' : [], '1965' : [], '1966' : [], '1967' : [], '1968' : [], '1969' : [], '1970' : [], '1971' : [], 
             '1972' : [], '1973' : [], '1974' : [], '1975' : [], '1976' : [], '1977' : [], '1978' : [], '1979' : [], '1980' : [], '1981' : [], '1982' : [], '1983' : [], '1984' : [], 
             '1985' : [], '1986' : [], '1987' : [], '1988' : [], '1989' : [], '1990' : [], '1991' : [], '1992' : [], '1993' : [], '1994' : [], '1995' : [], '1996' : [], '1997' : [], 
             '1998' : [], '1999' : [], '2000' : [], '2001' : [], '2002' : [], '2003' : [], '2004' : [], '2005' : [], '2006' : [], '2007' : [], '2008' : [], '2009' : [], '2010' : [], 
             '2011' : [], '2012' : [], '2013' : [], '2014' : [], '2015' : [], '2016' : [], '2017' : [], '2018' : [], '2019' : [], '2020' : [], '2021' : [], '2022' : [], '2023' : []}
for row in game.iterrows():
    season = str(row[1]["season"])
    game_dict[season].append(row[1]["game_id"])

In [7]:
player = pd.DataFrame(data = {"season" : [], 'person_id' : [], 'name' : [], 'field_goals_made' : [], 'field_goals_ast' : [], 'field_goals_missed' : [], 
                              'times_ball_lost' : [], 'times_ball_stolen' : [], 'fouls' : [], 'game_id' : [], 'games_played' : [], 'field_goals_blocked' : 0})
player

,season,person_id,name,field_goals_made,field_goals_ast,field_goals_missed,times_ball_lost,times_ball_stolen,fouls,game_id,games_played,field_goals_blocked


In [8]:
game_id = 0
count = 0
game_id_index = 0
eventmsgtype = 1
homedescription = 2
visitordescription = 3
player1_id = 6
player1_name = 7
player2_id = 8
player2_name = 9
player3_id = 10
player3_name = 11
print("starting...")
for key, values in game_dict.items():
    key = int(key)
    if key < 1996:
        continue
    print(key)
    for game in values:
        for row in play.loc[play["game_id"] == game].values:
            data = {"season" : key, 'person_id' : 0, 'name' : 0, 'field_goals_made' : 0, 'field_goals_ast' : 0, 'field_goals_missed' : 0, 'field_goals_blocked' : 0, 
                    'times_ball_lost' : 0, 'times_ball_stolen' : 0, 'fouls' : 0, 'game_id' : 0, 
                    'games_played' : 0}

            if (
                (row[eventmsgtype] == 8) | (row[eventmsgtype] == 9)
                | (row[eventmsgtype] == 10) | (row[eventmsgtype] == 11)
                | (row[eventmsgtype] == 12) | (row[eventmsgtype] == 13) 
                | (row[eventmsgtype] == 7) | pd.isna(row[player1_name]) | 
                (row[eventmsgtype] == 3) | (row[eventmsgtype] == 4)
            ):
                continue

            elif row[eventmsgtype] == 1:
                if not player.loc[((player["season"] == key) & (player["name"] == row[player1_name])), "name"].empty:
                    player.loc[player["name"] == row[player1_name], "field_goals_made"] += 1
                else:
                    data["person_id"] = row[player1_id]
                    data["name"] = row[player1_name]
                    data["field_goals_made"] = 1
                    player = player.append(data, ignore_index=True)

                if not pd.isna(row[player2_name]):
                    if not player.loc[((player["season"] == key) & (player["name"] == row[player2_name])), "name"].empty:
                        player.loc[player["name"] == row[player2_name], 'field_goals_ast'] += 1
                    else:
                        data["person_id"] = row[player2_id]
                        data["name"] = row[player2_name]
                        data["field_goals_ast"] = 1
                        player = player.append(data, ignore_index=True)

            elif row[eventmsgtype] == 2:
                if not player.loc[((player["season"] == key) & (player["name"] == row[player1_name])), "name"].empty:
                    player.loc[player["name"] == row[player1_name], "field_goals_missed"] += 1
                else:
                    data["person_id"] = row[player1_id]
                    data["name"] = row[player1_name]
                    data["field_goals_missed"] = 1
                    player = player.append(data, ignore_index=True)

                if not pd.isna(row[player3_name]):
                    if not player.loc[((player["season"] == key) & (player["name"] == row[player3_name])), "name"].empty:
                        player.loc[player["name"] == row[player3_name], "field_goals_blocked"] += 1
                    else:
                        data["person_id"] = row[player3_id]
                        data["name"] = row[player3_name]
                        data["field_goals_blocked"] = 1
                        player = player.append(data, ignore_index=True)
            
            elif row[eventmsgtype] == 5:
                if not player.loc[((player["season"] == key) & (player["name"] == row[player1_name])), "name"].empty:
                    player.loc[player["name"] == row[player1_name], "times_ball_lost"] += 1
                else:
                    data["person_id"] = row[player1_id]
                    data["name"] = row[player1_name]
                    player.loc[player["name"] == row[player1_name], "times_ball_lost"] = 1
                    player = player.append(data, ignore_index=True)
                    
                if not pd.isna(row[player2_name]):
                    if not player.loc[((player["season"] == key) & (player["name"] == row[player2_name])), "name"].empty:
                        player.loc[player["name"] == row[player2_name], "times_ball_stolen"] += 1
                    else:
                        data["person_id"] = row[player2_id]
                        data["name"] = row[player2_name]
                        player.loc[player["name"] == row[player2_name], "times_ball_lost"] = 1
                        player = player.append(data, ignore_index=True)
            
            elif row[eventmsgtype] == 6:
                if not player.loc[((player["season"] == key) & (player["name"] == row[player1_name])), "name"].empty:
                    player.loc[player["name"] == row[player1_name], "fouls"] += 1
                else:
                    data["person_id"] = row[player1_id]
                    data["name"] = row[player1_name]
                    player.loc[player["name"] == row[player1_name], "fouls"] = 1
                    player = player.append(data, ignore_index=True)
            
            if game_id != row[game_id_index]:
                game_id = row[game_id_index]
            
            if True in ((player["name"] == row[player1_name]).values.tolist()):
                if game_id != (player.loc[player["name"] == row[player1_name], "game_id"].values[0]):
                    player.loc[player["name"] == row[player1_name], "games_played"] += 1
                    player.loc[player["name"] == row[player1_name], "game_id"] = game_id
            if True in ((player["name"] == row[player2_name]).values.tolist()) and not pd.isna(row[player2_name]):
                if game_id != (player.loc[player["name"] == row[player2_name], "game_id"].values[0]):
                    player.loc[player["name"] == row[player2_name], "games_played"] += 1
                    player.loc[player["name"] == row[player2_name], "game_id"] = game_id
                    
            if count % 100000 == 0:
                print(count)
            count += 1
print("Finished")

starting...
1996
0
100000
200000
1997
300000
400000
500000
1998
600000
1999
700000
800000
900000
2000
1000000
1100000
1200000
2001
1300000
1400000
2002
1500000
1600000
1700000
2003
1800000
1900000
2000000
2004
2100000
2200000
2005
2300000
2400000
2500000
2006
2600000
2700000
2800000
2007
2900000
3000000
2008
3100000
3200000
3300000
2009
3400000
3500000
3600000
2010
3700000
3800000
3900000
2011
4000000
4100000
2012
4200000
4300000
2013
4400000
4500000
4600000
2014
4700000
4800000
4900000
2015
5000000
5100000
2016
5200000
5300000
5400000
2017
2018
5500000
5600000
5700000
2019
5800000
5900000
2020
6000000
6100000
6200000
2021
6300000
6400000
6500000
2022
6600000
6700000
2023
Finished


In [10]:
from pathlib import Path  
filepath = Path('Lumiere_Reasearch/newer_csv.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
player.to_csv(filepath) 